In [29]:
import os
import xml.etree.ElementTree as ET

path = '../data/korba_corpus/'

In [ ]:
for root, dirs, files in os.walk(path):
    if len(dirs) > 0:
        for directory in dirs:
            new_path = os.path.join(path, directory, 'ann_morphosyntax.xml')
            #print(new_path)
            tree = ET.parse(new_path)
            root = tree.getroot()
            for child in root.findall('seg'):
                print(child)